In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from sklearn.datasets import make_regression
from patsy import dmatrix
from sklearn.linear_model import LinearRegression
import statsmodels.stats.api as sms
import numpy as np

%matplotlib inline

/Users/jakekong/.pyenv/versions/3.6.1/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
orders = pd.read_csv('orders.csv')
orders.tail()

# order_id = order identification number
# user_id = user identification number
# order_number = how many times an user had made purchases
# order_dow = day of week when an order had been made (Mon - Sun by 0 -6)
# order_hour_of_day = hour of day when an oder had been made (1 - 23 in hours)
# days_since_prior_order = day interval from the previous purchase to the next purchase

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0
3421082,272231,206209,train,14,6,14,30.0


In [3]:
products = pd.read_csv('products.csv')
products.tail()

# product_id = product identification number
# product_name = product name
# aisle_id = aisle number
# department_id = department number 

,product_id,product_name,aisle_id,department_id
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8
49687,49688,Fresh Foaming Cleanser,73,11


In [4]:
aisles = pd.read_csv('aisles.csv')
aisles.tail()

# aisle (string) is matched with aisle_id (int)

,aisle_id,aisle
129,130,hot cereal pancake mixes
130,131,dry pasta
131,132,beauty
132,133,muscles joints pain relief
133,134,specialty wines champagnes


In [5]:
departments = pd.read_csv('departments.csv')
departments.tail()

# department (string) is matched with department_id (int)

,department_id,department
16,17,household
17,18,babies
18,19,snacks
19,20,deli
20,21,missing


# making a dataframe that conveys product_desc
* product_id
* product_name
* aisle, aisle_id
* department, departmnet_id

In [6]:
merged_data = pd.merge(left = (pd.merge(left = products, right = aisles)), right = departments)
merged_data.to_csv('merged_data.csv', index = False)

In [7]:
product_desc = pd.read_csv('merged_data.csv')

In [8]:
product_desc = product_desc.sort_values(['product_id'], ascending = True)
product_desc.tail()

,product_id,product_name,aisle_id,department_id,aisle,department
46968,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5,spirits,alcohol
19959,49685,En Croute Roast Hazelnut Cranberry,42,1,frozen vegan vegetarian,frozen
48383,49686,Artisan Baguette,112,3,bread,bakery
37921,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8,cat food care,pets
25484,49688,Fresh Foaming Cleanser,73,11,facial care,personal care


In [10]:
order_products__prior = pd.read_csv('order_products__prior.csv')
order_products__prior.tail()

# this dataframe is about the orders that happened in the older history

,order_id,product_id,add_to_cart_order,reordered
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1
32434488,3421083,5020,10,1


In [13]:
order_products__train = pd.read_csv('order_products__train.csv')
order_products__train.tail()

# it seems that this dataframe should be used to train the model

,order_id,product_id,add_to_cart_order,reordered
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1
1384616,3421070,4724,3,1


In [15]:
merged_data1 = pd.merge(left = order_products__train, right = product_desc)

In [17]:
merged_data1.sort_values('order_id')

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs
3627,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods
3651,1,13176,6,0,Bag of Organic Bananas,24,4,fresh fruits,produce
19131,1,47209,7,0,Organic Hass Avocado,24,4,fresh fruits,produce
26424,1,22035,8,1,Organic Whole String Cheese,21,16,packaged cheese,dairy eggs
1214,1,49683,4,0,Cucumber Kirby,83,4,fresh vegetables,produce
8,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs
152,1,10246,3,0,Organic Celery Hearts,83,4,fresh vegetables,produce
36714,36,46979,8,1,Asparagus,83,4,fresh vegetables,produce
28417,36,39612,1,0,Grated Pecorino Romano Cheese,2,16,specialty cheeses,dairy eggs


In [18]:
merged_data2 = pd.merge(left = order_products__prior, right = product_desc)

In [19]:
merged_data2.sort_values('order_id')

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,2,33120,1,1,Organic Egg Whites,86,16,eggs,dairy eggs
93148,2,45918,4,1,Coconut Butter,19,13,oils vinegars,pantry
93892,2,30035,5,0,Natural Sweetener,17,13,baking ingredients,pantry
94461,2,17794,6,1,Carrots,83,4,fresh vegetables,produce
86849,2,9327,3,0,Garlic Powder,104,13,spices seasonings,pantry
170810,2,43668,9,0,Classic Blend Cole Slaw,123,4,packaged vegetables fruits,produce
168386,2,1819,8,1,All Natural No Stir Creamy Almond Butter,88,13,spreads,pantry
167197,2,40141,7,1,Original Unflavored Gelatine Mix,105,13,doughs gelatins bake mixes,pantry
19400,2,28985,2,1,Michigan Organic Kale,83,4,fresh vegetables,produce
565721,3,32665,8,1,Organic Ezekiel 49 Bread Cinnamon Raisin,112,3,bread,bakery


### it seems that we should build a model based on the data, order_products__prior and order_products__train
* eval_set train will be our dependent feature while prior being our independent features

### then we can use the built model to predict order_products__test data
* similarly, we use prior data as our indepdent features to predict eval_set = test data